# 1. Introduction
First, we have to activate the environment. The first run may take a while as external libraries will be downloaded and compiled.

In [1]:
Pkg.activate(".")

  Activating project at `~/code/OpenScience21.jl`


In [2]:
#using QuantumCircuits.QCircuits.Qiskit
#using QuantumCircuits.QCircuits.Qiskit: qiskit

using QuantumCircuits
using QuantumCircuits.QML
using QuantumCircuits.QML.Optimization
using QuantumCircuits.QCircuits.Circuit
using QuantumCircuits.QCircuits.ComplexGates
using QuantumCircuits.Execute

# 2. The Algorithm
Define the backend, this is a simulator written in Julia. The main advantage of it is that the circuit executed on them can be automated differentiated by Zygote so each iteration of the QML algorithm takes less time (single execution of zygote vs execute whole circuits twice times that we have parameters for using shift-rules, in our case 27*2=54).

In [3]:
const backend = QuantumSimulator()

QuantumSimulator()

In [4]:
# The init file contains useful scripts and imports as well as connections to the IBM Qiskit account.
#include("src/Challenge/init.jl")

The method to the generation of the empty circuit.

In [5]:
function generate_empty_circuit(;init=false)
    qr = QuantumRegister(7, "q")
    qc = QCircuit(qr)

    # Prepare initial state (remember we are only evolving 3 of the 7 qubits on jakarta qubits (q_5, q_3, q_1) corresponding to the state |110>)
    if init
        qc.x([3, 5])  # DO NOT MODIFY (|q_5,q_3,q_1> = |110>)
    end

    return qc
end


generate_empty_circuit (generic function with 1 method)

The method to the generation of the ansact.

In [6]:
function generate_ansact()
    n = 3
    qr = QuantumRegister(7, "q")
    qc = QCircuit(qr)
    qr = [qr[1], qr[3], qr[5]]

    qc.u3(qr)
    for i in (n-2):-1:0
        i = i+1
        qc.cx(qr[i], qr[i+1])
        #qc.rzx(qr[i], qr[i+1])
    end
    qc.u3(qr)
    for i in 0:(n-2)
        i = i+1
        qc.cx(qr[i], qr[i+1])
        #qc.rzx(qr[i], qr[i+1])
    end
    qc.u3(qr)

    return qc
end

generate_ansact (generic function with 1 method)

The method adds measurement to the circuit.

In [7]:
function addMeasuresOS(qc)
    cr = ClassicalRegister(3)
    setClassicalRegister!(qc, cr)
    qc.measure([1, 3, 5], [0, 1, 2])
    nothing
end

addMeasuresOS (generic function with 1 method)

In [8]:
# This is method from QuantumCircuits.Execute
"Loss method to check if final state is zero."
function loss_expected_zero_state(state)
    return -log(real(state[1])+1e-32) + sum([real(v)^2 for v in state[2:end]])
end


loss_expected_zero_state

The next function finds the best ansac parameters, the main steps are:
1. Generation of the empty circuit
1. Generation of the ansact
1. Append both to single circuit
1. Add measure
1. Generate rand parameters
1. Define loss function, it has a minimum equal to 0 when the executed state is $|000\rangle$
1. Using Zygote, we define the gradient
1. Using the gradient descent method Eva (see [06_Optimization](06_Optimization.ipynb)) we try to find the best parameters.

In [9]:
function generate_ansact_param(step_qc_exp, maxItr=1000)
    # Generate step qc
    step_qc = generate_empty_circuit()
    ansact = generate_ansact()
    # append
    append!(step_qc, step_qc_exp)
    append!(step_qc, ansact)
    # Add measures
    addMeasuresOS(step_qc)

    # Set random parameters
    start_params = getRandParameters(step_qc)
    setparameters!(ansact, start_params)


    loss(params) = loss_expected_zero_state(execute(backend, step_qc, params))
    dloss(params) = real(loss'(params))
    of = OptimizationFunction(false, (x) -> (loss(x), dloss(x)), loss)

    val, xparams, itr = gradientDescent(of, start_params, α=0.01, maxItr=maxItr,
                                  argsArePeriodic=true, isExpectedZero=true, ϵ=1e-4, debug=false, useBigValInc=true)

    return xparams
end

generate_ansact_param (generic function with 2 methods)

We define the Trotter step number and the number of trotter steps used in a single optimization process. I always use 2, but this may be changed and the method will work properly. There is one point to note, when we use more trotter step in a single iteration there will be less number of the iteration and so one whole algorithm runs quicker from the other hand less number of trotter step work better on real devices (the circuit has less depth and a smaller number of cx gates). For a description of the algorithm look at the notebook [01_solution](01_solution.ipynb).

In [10]:
trotter_steps = 10
trotter_steps_arr = [2, 2, 2, 2, 2]
nothing

The code below is used to generate the trotter circuit, you can find the description of it in the notebook [03_trotter_step](03_trotter_step.ipynb).

In [11]:
# This code com from using OpenScience21.Simulation.Gates module.
function ZZ(qc, q0, q1, t, usePulse=false)
    if usePulse
        qc.h(q1)
        qc.rzx(q0, q1, t)
        qc.x(q0)
        qc.rzx(q0, q1, -t)
        qc.x(q0)
        qc.h(q1)
    else
        qc.cx(q0, q1)
        qc.rz(q1, 2*t)
        qc.cx(q0, q1)
    end
end

function YY(qc, q0, q1, t, usePulse=false)
    if usePulse
        qc.sdg([q0, q1])
        qc.h(q0)
        qc.rzx(q0, q1, t)
        qc.x(q0)
        qc.rzx(q0, q1, -t)
        qc.x(q0)
        qc.h(q0)
        qc.s([q0, q1])
    else
        qc.rx([q0, q1], π/2)
        qc.cx(q0, q1)
        qc.rz(q1, 2*t)
        qc.cx(q0, q1)
        qc.rx([q0, q1], -π/2)
    end
end

function XX(qc, q0, q1, t, usePulse=false)
    if usePulse
        qc.h(q0)
        qc.rzx(q0, q1, t)
        qc.x(q0)
        qc.rzx(q0, q1, -t)
        qc.x(q0)
        qc.h(q0)
    else
        qc.ry([q0, q1], π/2)
        qc.cx(q0, q1)
        qc.rz(q1, 2*t)
        qc.cx(q0, q1)
        qc.ry([q0, q1], -π/2)
    end
end

# This code com from using OpenScience21.Simulation.Gates module.
function findU4paramsZZYYXX(t; debug=false)
    qc = QCircuit(2)
    ZZ(qc, 0, 1, t)
    YY(qc, 0, 1, t)
    XX(qc, 0, 1, t)
    expmat = tomatrix(qc)


    qr = QuantumRegister(2)
    qc = QCircuit(qr)
    qc.u4(qr[0], qr[1])

    params = getRandParameters(qc)
    setparameters!(qc, params)
    qc = decompose(qc)

    params, _, err, _  = findparam(expmat, qc, debug=debug, trystandard=false)

    @assert err < 1e-5 "The error of U gate should be small but it is $err."

    return params
end

function findU4paramsZZYYXXx2(t; debug=false)
    qc = QCircuit(2)
    ZZ(qc, 0, 1, t)
    YY(qc, 0, 1, t)
    XX(qc, 0, 1, t)
    ZZ(qc, 0, 1, t)
    YY(qc, 0, 1, t)
    XX(qc, 0, 1, t)
    expmat = tomatrix(qc)


    qr = QuantumRegister(2)
    qc = QCircuit(qr)
    qc.u4(qr[0], qr[1])

    params = getRandParameters(qc)
    setparameters!(qc, params)
    qc = decompose(qc)

    params, _, err, _  = findparam(expmat, qc, debug=debug, trystandard=false)
    @assert err < 1e-5 "The error of U gate should be small but it is $err."

    return params
end

function trotter2U4(qc, qubits, t, isFirst, isLast, params, params2, params3)
    if isFirst
        for i in 1:(length(qubits)-2)
            qc.u4(qubits[i], qubits[i+1], params2)
        end
    end

    i = length(qubits) - 1
    qc.u4(qubits[i], qubits[i+1], params)

    if isLast
        for i in 1:(length(qubits)-2)
            qc.u4(qubits[i], qubits[i+1], params2)
        end
    else
        for i in 1:(length(qubits)-2)
            qc.u4(qubits[i], qubits[i+1], params3)
        end
    end
end

function generate_circuit(trotter_steps, run_step, t=π, params=nothing, params2=nothing, params3=nothing; init=false, debug=false)
    qr = QuantumRegister(7, "q")
    qc = QCircuit(qr)

    # Prepare initial state (remember we are only evolving 3 of the 7 qubits on jakarta qubits (q_5, q_3, q_1) corresponding to the state |110>)
    if init
        qc.x([3, 5])  # DO NOT MODIFY (|q_5,q_3,q_1> = |110>)
    end

    if params == nothing
        params = findU4paramsZZYYXX(t / trotter_steps, debug=debug)
        params2 = findU4paramsZZYYXX(t / (2 * trotter_steps), debug=debug)
        params3 = findU4paramsZZYYXXx2(t / (2 * trotter_steps), debug=debug)
    end

    qubits = [qr[1], qr[3], qr[5]]
    for s in 1:run_step
        isFirst = s == 1
        isLast = s == run_step

        trotter2U4(qc, qubits, t / trotter_steps, isFirst, isLast, params, params2, params3)
    end

    qc = decompose(qc)
    return qc, params, params2, params3
end

using QuantumCircuits.QCircuits.Math
using QuantumCircuits.QCircuits.Gates: Xmatrix, Ymatrix, Zmatrix

XXs = kron(kron(eye(2), Xmatrix), Xmatrix) + kron(kron(Xmatrix, Xmatrix), eye(2))
YYs = kron(kron(eye(2), Ymatrix), Ymatrix) + kron(kron(Ymatrix, Ymatrix), eye(2))
ZZs = kron(kron(eye(2), Zmatrix), Zmatrix) + kron(kron(Zmatrix, Zmatrix), eye(2))
Hs = XXs + YYs + ZZs

U_heis3(t) = exp(-im * Hs * t)

function check_simulation_err(qc, t)
    sym_full = execute(backend, qc)
    exp_full = U_heis3(t) * ket"110"
    exp_full = abs.(exp_full) .^ 2
    return sum(abs.(sym_full - exp_full))
end

check_simulation_err (generic function with 1 method)

The below method compares the results of the execution of two circuits.

In [12]:
function check_circuits_err(qc1, qc2)
    sym1 = execute(backend, qc1)
    sym2 = execute(backend, qc2)

    sum(abs.(sym1 - sym2))
end

check_circuits_err (generic function with 1 method)

# 3. The Evaluation
We are now putting all the pieces together. For all simulation times which we would like to check we do the whole procedure:
* For the first step, we generate a circuit with 2 trotter steps and the ansact. And we find the parameters that the output state is $|000\rangle$.
* For the next steps, we generate a circuit with the inverse of the ansact from the previous step, 2 trotter steps, and the new ansact. And we find the parameters that the output state is $|000\rangle$.

I use exactly the same ansact for all iterations but when can have different ansacts for other iterations. It is quite easy to check if the found parameters are properly. If the output state for a given iteration is near the $|000\rangle$ then we can go to the next step.

In [13]:
errs = []
ts = [π/4, π/2, 3*π/4, π]
#ts = [π]
best_param = []
best_qc = nothing
for t in ts
    println("Start for t=$t")
    qc, params, params2, params3 = generate_circuit(trotter_steps, trotter_steps, t)

    full_qc = generate_empty_circuit()
    full_qc.x([3, 5])
    append!(full_qc, qc)
    addMeasuresOS(full_qc)

    ###########################################################################
    do_iter = 0
    do_check = true
    first_step = true
    inv_ansact = []
    for st in trotter_steps_arr
        step_qc_exp, _, _, _ = generate_circuit(trotter_steps, st, t, params, params2, params3, init=first_step)
        #step_qc_expmat = tomatrix(step_qc)
        do_iter += st
        println("Start AI TS $do_iter")

        if first_step
            opt_step_qc_exp = step_qc_exp
        else
            opt_step_qc_exp = generate_empty_circuit()
            append!(opt_step_qc_exp, inv_ansact)
            append!(opt_step_qc_exp, step_qc_exp)
        end

        # find best parameters
        best_params = generate_ansact_param(opt_step_qc_exp, 10000)

        # Generate ansact
        ansact = generate_ansact()
        setparameters!(ansact, best_params)
        inv_ansact = inv(ansact)
        bindparameters!(inv_ansact)

        # Check step reaults
        if do_check && do_iter < trotter_steps
            check_qc, _, _, _ = generate_circuit(trotter_steps, trotter_steps-do_iter, t, params, params2, params3)

            # Simulate full circuit
            check_step_qc_full = generate_empty_circuit()
            append!(check_step_qc_full, inv_ansact)
            append!(check_step_qc_full, check_qc)
            # Add measures
            addMeasuresOS(check_step_qc_full)

            # do check
            check_err = check_circuits_err(full_qc, check_step_qc_full)
            println("Check value: $check_err")
            @assert check_err < 10e-2
        end

        # next initmat
        first_step = false
    end

    ###########################################################################
    addMeasuresOS(inv_ansact)
    check_err = check_circuits_err(full_qc, inv_ansact)
    append!(errs, check_err)
end

Start for t=0.7853981633974483
After iteration 1000, value: 0.0005443169443671981, best: 0.0004795506699154643, dval: 0.004959093947253287, H: 1.6656155474379078e8, α: 0.6495514041771517, diffx: 0.002919522838916222
After iteration 2000, value: 1.7596711537268157, best: 2.282472757415722e-5, dval: 2.437504343499535, H: 117626.06634120029, α: 12.07315509441982, diffx: 3.631353867799774
After iteration 3000, value: 4.260737393445782e-6, best: 4.260737393445782e-6, dval: 0.0008922164520049825, H: 142.11180213618434, α: 0.8845737200114944, diffx: 7.210214038011475e-5
After iteration 4000, value: 0.00040012627347736987, best: 1.9462701451067405e-6, dval: 0.029762902416208936, H: 156.8194366334606, α: 0.5087869760592589, diffx: 0.0012575677833952525
Start AI TS 2
Check value: 0.008170403227342105
Start AI TS 4
Check value: 0.012503386217069118
Start AI TS 6
Check value: 0.01272394945455379
Start AI TS 8
Check value: 0.013312241174504833
Start AI TS 10
Start for t=1.5707963267948966
Start AI 

The error of the method. Please note that we can improve the error when we will use more iterations to find the best parameters.

In [14]:
println("==============================")
println("The error is $(sum(errs)).")
println("==============================")

The error is 0.02277650357248752.


In [15]:
errs

4-element Vector{Any}:
 0.013550464170137515
 0.005693337607823896
 0.0024389045697131686
 0.0010937972248129387